# Bluesky sample stage operation

Cleaning up bluesky routines and testing sample stage scan before commissioning

In [2]:
# October 2024
# awojdyla@lbl.gov, xchong@lbl.gov, francisho@lbl.gov

In [1]:
import epics
import bluesky
#import numpy
#import matplotlib.pyplot as plt
#import time 

In [2]:
from bluesky import RunEngine
RE = RunEngine({})

In [3]:
from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

0

### Connecting to EPICS motors
We had some issues here witht the picoscale interformeter -- we need to use caproto to talk to it (ophyd's motor does not work directly out of the box)

from: https://nsls-ii.github.io/bluesky/tutorial.html

```
from ophyd import EpicsMotor
nano_top_x = EpicsMotor('XF:31ID-ES{Dif:Nano-Ax:TopX}Mtr', name='nano_top_x')
```
see also:

https://blueskyproject.io/ophyd/generated/ophyd.epics_motor.EpicsMotor.html#:~:text=class%20ophyd.epics_motor.EpicsMotor%28prefix%3D%27%27%2C%20%2A%2C%20name%2C%20kind%3DNone%2C%20read_attrs%3DNone%2C%20configuration_attrs%3DNone%2C%20parent%3DNone%2C,are%20passed%20through%20to%20the%20base%20class%2C%20Positioner




In [4]:
from ophyd import EpicsMotor
import ophyd
ophyd.set_cl('caproto')

sample_x_mm = ophyd.EpicsMotor('bl531_xps2:sample_x_mm', name='sample stage horizontal')
sample_y_mm = ophyd.EpicsMotor('bl531_xps2:sample_y_mm', name='sample stage vertical')
beamstop_diode = ophyd.EpicsSignal('bl201-beamstop:current', name='beamstop diode')

#this works
sample_x_mm.wait_for_connection(timeout=2.0)
sample_x_mm.read()
sample_y_mm.wait_for_connection(timeout=2.0)
sample_y_mm.read()
beamstop_diode.wait_for_connection(timeout=2.0)
beamstop_diode.read()

{'beamstop diode': {'value': 0.0, 'timestamp': 631152000.0}}

In [5]:
from bluesky.plans import scan
# just a motor scan, no detectors
RE(scan([], sample_x_mm, 1, 2, sample_y_mm, 1,2, 11))

#works, but does a weird scan
#https://nsls-ii.github.io/bluesky/generated/bluesky.plans.grid_scan.html#bluesky.plans.grid_scan


('f4442c1a-1e11-4583-8f5f-dbfb98ea40e3',)

In [7]:
from bluesky.plans import grid_scan

RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f1183da2-0559-4b45-9fac-280258be189d',)

In [6]:
from bluesky.plans import grid_scan

RE(grid_scan([beamstop_diode], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('d4536f2f-24d6-4115-83d0-a1f2d4851b01',)

In [7]:
header = db[-1]
header.table()

,time,beamstop diode,sample stage horizontal,sample stage horizontal_user_setpoint,sample stage vertical,sample stage vertical_user_setpoint
seq_num,,,,,,
1,2025-02-25 18:34:26.733349085,0.0,1.0,1.0,1.0,1.0
2,2025-02-25 18:34:28.219246864,0.0,1.0,1.0,1.5,1.5
3,2025-02-25 18:34:29.700576067,0.0,1.0,1.0,2.0,2.0
4,2025-02-25 18:34:31.878351688,0.0,1.5,1.5,1.0,1.0
5,2025-02-25 18:34:33.361911058,0.0,1.5,1.5,1.5,1.5
6,2025-02-25 18:34:34.838891983,0.0,1.5,1.5,2.0,2.0
7,2025-02-25 18:34:37.007189512,0.0,2.0,2.0,1.0,1.0
8,2025-02-25 18:34:38.487745762,0.0,2.0,2.0,1.5,1.5
9,2025-02-25 18:34:39.976714849,0.0,2.0,2.0,2.0,2.0


In [17]:
# some issues here -- not critical
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
# Send all metadata/data captured to the BestEffortCallback.
bec_token = RE.subscribe(bec)
# Make plots update live while scans run.
from bluesky.utils import install_kicker


/usr/local/lib/python3.10/dist-packages/bluesky/utils/__init__.py:934: UserWarning: bluesky.utils.install_qt_kicker is no longer necessary and has no effect. Please remove your use of it. It may be removed in a future release of bluesky.
  warnings.warn("bluesky.utils.install_qt_kicker is no longer necessary and "


In [21]:
RE.unsubscribe(bec_token)
RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f59ba064-15f1-4e77-a008-19e88331a6fc',)

In [ ]:
RE(scan(dets, motors, 25, 28, 61))

In [8]:
epics.caput('DMC02:E.CNEN', 0)
epics.caput('DMC02:F.CNEN', 0)
epics.caput('DMC02:G.CNEN', 0)
epics.caput('DMC02:H.CNEN', 0)

1